In [2]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [3]:
TRAIN_DIR= 'images/train'
TEST_DIR= 'images/test'

In [4]:
#From the images provided in the dataset we will create a data frame.
# columns = image, label

def createDataFrame(dir):
    '''
        directory denge train aur test ki.
        jo bhi folder hai train(say) ke andar unko list kar dega say angry, sad, happy, etc.,
        We will treat each folder as a label.
        This happens in label for loop.

        Next in the imagename for loop, each image in the labelled folder is traversed through.
        That image name is added to image_paths. So image_paths stores the whole path of the image.
        
        
    '''
    image_paths= []
    labels=[]
    for label in os.listdir(dir): #the dir is basically the path till images. os.listdir will find the contents of the image folder. Label will store the sub-folders present in image folder. 
        for imagename in os.listdir( os.path.join(dir,label)): #traverses each image present in the label folder. Say traverse each image present in the angry folder.
            image_paths.append(os.path.join(dir,label,imagename)) #a path is created .../image/angry/img0013
            labels.append(label) # in labels list, the particular label is appended. Say angry. So label stores all the emotions present.
        print(label," added to DataFrame")
    return image_paths,labels #image_paths contain the full path, labels contain the emotion.


In [5]:
train= pd.DataFrame()
train['image'], train['label'] = createDataFrame(TRAIN_DIR)

angry  added to DataFrame
disgust  added to DataFrame
fear  added to DataFrame
happy  added to DataFrame
neutral  added to DataFrame
sad  added to DataFrame
surprise  added to DataFrame


In [6]:
train #all the training images'-paths and their respective labels are stored in the train dataframe.

,image,label
0,images/train\angry\0.jpg,angry
1,images/train\angry\1.jpg,angry
2,images/train\angry\10.jpg,angry
3,images/train\angry\10002.jpg,angry
4,images/train\angry\10016.jpg,angry
...,...,...
28816,images/train\surprise\9969.jpg,surprise
28817,images/train\surprise\9985.jpg,surprise
28818,images/train\surprise\9990.jpg,surprise
28819,images/train\surprise\9992.jpg,surprise


In [7]:
test= pd.DataFrame()
test['image'], test['label'] = createDataFrame(TEST_DIR)

angry  added to DataFrame
disgust  added to DataFrame
fear  added to DataFrame
happy  added to DataFrame
neutral  added to DataFrame
sad  added to DataFrame
surprise  added to DataFrame


In [8]:
test

,image,label
0,images/test\angry\10052.jpg,angry
1,images/test\angry\10065.jpg,angry
2,images/test\angry\10079.jpg,angry
3,images/test\angry\10095.jpg,angry
4,images/test\angry\10121.jpg,angry
...,...,...
7061,images/test\surprise\9806.jpg,surprise
7062,images/test\surprise\9830.jpg,surprise
7063,images/test\surprise\9853.jpg,surprise
7064,images/test\surprise\9878.jpg,surprise


In [9]:
from tqdm.notebook import tqdm

In [10]:
def extractFeatures(images): 
    '''
    Now we will pick apart each image and perform feature extraction on them.
    The images being received as input to the function will contain dataFrame.image column only i.e., all the paths of the images.
    The tqdm module helps in accessing all the images that are present in the image path. 
    For loop wala image only contains the path that have been mentioned in the 
    images list. To see the progress of each image present int the path, we use tqdm module.
    '''
    features=[] #the features extracted through the image will be put in this list.
    for image in tqdm(images): #will traverse every image whose path have been provided in the images.
        img= load_img(image, grayscale=True) #converted the image to grayscale as it takes only 1 channel for processing whereas the classic RGB takes 3 channels that at times can become complex.
        img = np.array(img) #image is converted to array
        features.append(img) 
    features= np.array(features) #convert features to array.
    features= features.reshape(len(features), 48,48,1) #length, image width,image height, image depth=1 as its 2D
    return features

    
    


In [11]:
train_features= extractFeatures(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

C:\Users\Lucky\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [12]:
test_features= extractFeatures(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [13]:
x_train = train_features/255.0 #(divided by highest pixel value = 255)
x_test= test_features/255.0

In [14]:
#to make label we use labelEncoder
from sklearn.preprocessing import LabelEncoder

In [15]:
le= LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [16]:
y_train= le.transform(train['label'])
y_test= le.transform(test['label'])

In [17]:
y_train= to_categorical(y_train, num_classes= 7)
y_test= to_categorical(y_test, num_classes=7)

In [18]:
model= Sequential()

#cnn layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer, 7= number of classes
model.add(Dense(7, activation='softmax'))


C:\Users\Lucky\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
#model.compile(optimizer= 'adam', loss= 'categorical_crossentropy', metrics= 'accuracy')\

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [ ]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data= (x_test, y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 75s 324ms/step - accuracy: 0.2351 - loss: 1.8424 - val_accuracy: 0.2605 - val_loss: 1.8129
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 95s 419ms/step - accuracy: 0.2481 - loss: 1.8107 - val_accuracy: 0.2927 - val_loss: 1.7267
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 92s 407ms/step - accuracy: 0.2885 - loss: 1.7371 - val_accuracy: 0.3576 - val_loss: 1.6177
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 92s 408ms/step - accuracy: 0.3447 - loss: 1.6408 - val_accuracy: 0.4369 - val_loss: 1.4898
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 91s 403ms/step - accuracy: 0.4058 - loss: 1.5289 - val_accuracy: 0.4564 - val_loss: 1.3983
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 93s 410ms/step - accuracy: 0.4181 - loss: 1.4826 - val_accuracy: 0.4979 - val_loss: 1.3183
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 91s 403ms/step - accuracy: 0.4530 - loss: 1.4180 - val_accuracy: 0.5170 - val_loss: 1.2658
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 94s 415ms/step - accuracy: 0.4777 -

In [ ]:
model_json= model.to_json()
with open("emotiondetector.json","w") as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")


In [ ]:
from keras.models import model_from_json


In [ ]:
json_file= open("emotiondetector.json","r")
model_json= json_file.read()
json_file.close()
model= model_from_json(model_json)
model.load_weights("emotiondetector.h5")


In [ ]:
label= ['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
def ef(image):
    img= load_img(image,grayscale= True)
    feature=np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [ ]:
image='images/train/angry/27.jpg'
print("OG is ANGRY")
img= ef(image)
pred= model.predict(img)
pred_label= label[pred.argmax()]
print("Model predicted : ", pred_label)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image='images/train/angry/10.jpg'
print("OG is ANGRY")
img= ef(image)
pred= model.predict(img)
pred_label= label[pred.argmax()]
print("Model predicted : ", pred_label)
plt.imshow(img.reshape(48,48), cmap="gray")